In [ ]:
import os
user = 'layakarimichahrogh'
key = '58cd771a8bf296499cfc57dda07fa8d0'
if '.kaggle' not in os.listdir('/root'):
    !mkdir ~/.kaggle
!touch /root/.kaggle/kaggle.json
!chmod 666 /root/.kaggle/kaggle.json
with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write('{"username":"%s","key":"%s"}' % (user, key))
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d sudalairajkumar/novel-corona-virus-2019-dataset

100% 4.88M/4.88M [00:00<00:00, 13.2MB/s]



In [ ]:
!unzip '/content/novel-corona-virus-2019-dataset.zip'

Archive:  /content/novel-corona-virus-2019-dataset.zip
  inflating: COVID19_line_list_data.csv  
  inflating: COVID19_open_line_list.csv  
  inflating: covid_19_data.csv       
  inflating: time_series_covid_19_confirmed.csv  
  inflating: time_series_covid_19_confirmed_US.csv  
  inflating: time_series_covid_19_deaths.csv  
  inflating: time_series_covid_19_deaths_US.csv  
  inflating: time_series_covid_19_recovered.csv  


In [ ]:
import pandas as pd
I=pd.read_csv ("/content/time_series_covid_19_confirmed.csv")


In [ ]:
R=pd.read_csv("/content/time_series_covid_19_deaths.csv")


In [ ]:
import numpy as np 

usi=I.loc[I["Country/Region"]=="US"]
usi=np.array(usi)
usi = usi.T.reshape(usi.T.shape[0],)
usi=usi[4:]
usi.shape

(320,)

In [ ]:
import numpy as np

usd=R.loc[R["Country/Region"]=="US"]
usd=np.array(usd)
usd = usd.T.reshape(usd.T.shape[0],)
usd=usd[4:]
usd.shape

(320,)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

import plotly.express as px

y=list(range(usi.shape[0]))

fig = px.line()
fig.add_scatter(y=usd,x=y,mode='lines',name='US Death')
fig.update_yaxes(type="log")
fig.add_scatter(y=usi,x=y,name='US Infection')

fig.show()

In [ ]:
nd = 60


xi = []
yi = []

for i in range(nd,len (usi)):
  xii = []
  for j in range(nd):
    xii.append(usi[i-nd+j])
  xi.append(xii)
  yi.append(usi[i])


xd = []
yd = []

for i in range(nd,len (usd)):
  xdd = []
  for j in range(nd):
    xdd.append(usd[i-nd+j])
  xd.append(xdd)
  yd.append(usd[i])

In [ ]:
print(len(xd[0]))
print(xd[0])
print(yd[0])

60
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 6, 7, 11, 12, 14, 17, 21, 22, 28, 33, 43, 51, 58, 70, 97, 132, 191, 264, 365, 464]
574


In [ ]:
xi = np.array(xi)
xd = np.array(xd)
yi = np.array(yi)
yd = np.array(yd)

xi = xi.astype(float)
yi = yi.astype(float)
xd = xd.astype(float)
yd = yd.astype(float)

print(xi.shape)
print(xd.shape)
print(yi.shape)
print(yd.shape)

(260, 60)
(260, 60)
(260,)
(260,)


In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *

m = Sequential()

m.add(Dense(1000, activation='relu'))
m.add(Dense(100, activation='relu'))
m.add(Dense(10, activation='relu'))
m.add(Dense(1,activation='linear'))

m.compile(optimizer='adam',loss='mse', metrics='mae')

def fit_model(model):
    early_stop = EarlyStopping(monitor = 'loss',
                                               patience = 10)
    history = model.fit(xi, yi, epochs = 200,  
                        validation_split = 0.1, batch_size = 1, 
                        shuffle = False, callbacks = [early_stop])
    return history

fit_model(m)

Epoch 1/200
234/234 [==============================] - 0s 2ms/step - loss: 25245054976.0000 - mae: 103454.7891 - val_loss: 173846478848.0000 - val_mae: 357614.8750
Epoch 2/200
234/234 [==============================] - 0s 1ms/step - loss: 7572399104.0000 - mae: 52913.7148 - val_loss: 633791905792.0000 - val_mae: 686507.3750
Epoch 3/200
234/234 [==============================] - 0s 1ms/step - loss: 3901355520.0000 - mae: 29517.2324 - val_loss: 1256213577728.0000 - val_mae: 1029085.6875
Epoch 4/200
234/234 [==============================] - 0s 1ms/step - loss: 8745114624.0000 - mae: 53900.8750 - val_loss: 754157813760.0000 - val_mae: 755844.3750
Epoch 5/200
234/234 [==============================] - 0s 1ms/step - loss: 2063010560.0000 - mae: 27810.5898 - val_loss: 1568931708928.0000 - val_mae: 1163960.5000
Epoch 6/200
234/234 [==============================] - 0s 1ms/step - loss: 18432258048.0000 - mae: 83109.7266 - val_loss: 45885005824.0000 - val_mae: 180619.3438
Epoch 7/200
234/234 [=

In [ ]:
yp = m.predict(xi)
yp = yp.reshape(yp.shape[0],)
import plotly.express as px
import plotly.graph_objects as go

import plotly.express as px

y=list(range(usi.shape[0]))

fig = px.line()
fig.add_scatter(y=yp,mode='lines',name='US Infection Prediction')
fig.update_yaxes(type="log")
fig.add_scatter(y=yi,name='US Infection True')

fig.show()

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *

m = Sequential()

m.add(Dense(1000, activation='relu'))
m.add(Dense(100, activation='relu'))
m.add(Dense(10, activation='relu'))
m.add(Dense(1,activation='linear'))

m.compile(optimizer='adam',loss='mse', metrics='mae')

def fit_model(model):
    early_stop = EarlyStopping(monitor = 'loss',
                                               patience = 10)
    history = model.fit(xd, yd, epochs = 200,  
                        validation_split = 0.1, batch_size = 1, 
                        shuffle = False, callbacks = [early_stop])
    return history

fit_model(m)

Epoch 1/200
234/234 [==============================] - 0s 2ms/step - loss: 16837080.0000 - mae: 3267.6533 - val_loss: 7791086.0000 - val_mae: 2240.2339
Epoch 2/200
234/234 [==============================] - 0s 1ms/step - loss: 8306272.5000 - mae: 1903.4401 - val_loss: 31041106.0000 - val_mae: 4572.3652
Epoch 3/200
234/234 [==============================] - 0s 1ms/step - loss: 6056837.0000 - mae: 1764.2311 - val_loss: 31575026.0000 - val_mae: 4722.4238
Epoch 4/200
234/234 [==============================] - 0s 1ms/step - loss: 7498625.0000 - mae: 1676.3292 - val_loss: 106633008.0000 - val_mae: 9961.8496
Epoch 5/200
234/234 [==============================] - 0s 1ms/step - loss: 4000173.2500 - mae: 1524.6755 - val_loss: 261443728.0000 - val_mae: 15617.3320
Epoch 6/200
234/234 [==============================] - 0s 1ms/step - loss: 13345139.0000 - mae: 2584.8389 - val_loss: 392078528.0000 - val_mae: 19556.6914
Epoch 7/200
234/234 [==============================] - 0s 1ms/step - loss: 1446385

In [ ]:
yp = m.predict(xd)
yp = yp.reshape(yp.shape[0],)
import plotly.express as px
import plotly.graph_objects as go

import plotly.express as px

y=list(range(usi.shape[0]))

fig = px.line()
fig.add_scatter(y=yp,mode='lines',name='US Death Prediction')
fig.update_yaxes(type="log")
fig.add_scatter(y=yd,name='US Death True')

fig.show()

In [ ]:
m.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (1, 1000)                 61000     
_________________________________________________________________
dense_41 (Dense)             (1, 100)                  100100    
_________________________________________________________________
dense_42 (Dense)             (1, 10)                   1010      
_________________________________________________________________
dense_43 (Dense)             (1, 1)                    11        
Total params: 162,121
Trainable params: 162,121
Non-trainable params: 0
_________________________________________________________________
